# Common

In [1]:
import numpy as np
import pandas as pd
from tqdm.contrib.concurrent import process_map
#from tqdm import tqdm  # !pip install tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
#from tqdm import trange
import multiprocessing as mp
import shutil
import glob
import os
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [2]:
!mkdir -p Temp

In [3]:
def bracket_row(row):    
    s = row['data']
    index = min(s.find('.'), s.find('('))
    data = row['data']
    row['data'] = data[0:index]
    row['bracket'] = data[index:]
    return row

In [4]:
def adjust(text,n=7):
    text = str(text)    
    return " " * (n - len(text)) + text

In [5]:
def bracket_to_ct(tag, data, bracket, deltaG, negative_deltaG=True):    
    deltaG = deltaG.replace('(','').replace(')','')
    deltaG = float(deltaG)
    if(deltaG > 0 and negative_deltaG ): # negetive?!
        deltaG = -1 * deltaG
    stack = []
    index = np.zeros((len(bracket)), dtype = int)
    values = np.zeros((len(bracket)), dtype = int)
    for i in range(len(bracket)):
        index[i] = i + 1
        if(bracket[i] == '.'):
            values[i] = 0
        elif(bracket[i] == '('):
            stack.append(i)
        elif(bracket[i] == ')'):
            if(len(stack) == 0 ):
                print('structure error!')
            values[stack[-1]] = i + 1
            values[i]  = stack[-1] + 1
            stack.pop()
        else:
            print('structure error!')
    if(len(stack) != 0 ):
        print('structure error!')
    # body    
    ct = f"{adjust(len(data),6)} dG ={adjust(deltaG,10)} {tag}\n"   
    for i in range(len(bracket)):
        ct += f"{adjust(index[i],6)} {data[i]} {adjust(i,6)} {adjust((i+2)%(len(data)+1),6)} {adjust(values[i],6)} {adjust(index[i],7)}\n"
    return ct

In [6]:
def fasta_to_df(path):
    with open(path, 'r') as file:
        text = file.read()
    lines = [line for line in text.split('\n') if len(line) > 0]
    s = ''
    tags = []
    data = []
    for l in lines:
        if(l[0]=='>'):
            tags.append(l)        
            data.append(s)
            s = ''
        else:
            s += l    
    data.append(s)
    df = pd.DataFrame(
            {
                'tag': tags,
                'data': data[1:]
            })
    df['tag'] = df['tag'].apply(lambda x: x[1:])    
    return df

In [7]:
def df_to_fasta(df, path):
    lines = []
    df.apply(lambda row: lines.append(f">{row['tag']}\n{row['data']}\n"),axis=1)
    with open(path,'w') as file:
        file.write(''.join(lines))

In [8]:
def reformat(path):
    return path.replace('(','_').replace(')','_').replace('.','').replace(':','_')

In [9]:
def reformatCT(path):
    with open(path, 'r') as file:
        text = file.read()
    text = [l for l in text.split('\n') if len(l) > 0 ] # remove blank lines
    text = '\n'.join(text)
    text = text.replace("\t"," ")
    while("  " in text):
        text = text.replace("  ", " ")
    lines = [l for l in text.split('\n')]
    for i in range(len(lines)):
        if(lines[i][0] == " "):
            lines[i] = lines[i][1:]
        if(lines[i][-1] == " "):
            lines[i] = lines[i][:-1]
    text = '\n'.join(lines)
    return text

In [10]:
def get_ct_data(ct):
    ct = "\n".join(ct.split('\n')[1:])
    df = pd.read_csv(StringIO(ct), sep=" ", header=None)               
    nucleotide = df.iloc[:,1]
    index = df.iloc[:,5]
    values = df.iloc[:,4]
    return [nucleotide, index, values]

In [11]:
def ct2dot_bracket(path):
    [nucleotide, index, values] = get_ct_data(reformatCT(path))
    text = ''.join(nucleotide) + "\n"
    watch = []
    for i, v in zip(index,values):
        if(v == 0):
            text += '.'
        else:
            if( v not in watch):
                text += '('
                watch.append(i)
            if( v in watch):
                text += ')'
    return text

In [12]:
def is_nested(index, values):
    max_value = max(index) + 10 # inf
    for i, v in zip(index, values):
        if(v < max_value and v != 0):
            max_value  = v
        if(i >= max_value):
            max_value = max(index) + 10 # inf
        if(v > max_value):
            return False               
    return True

In [13]:
'''ct = reformatCT('./secondary_structure/spot_rna/AMWY020598281_2832-3256_+_/AMWY020598281_2832-3256_+_.ct')
[nucleotide, index, values] = get_ct_data(ct)
print(is_nested( index,  values))
''';

### rename tag of input genome to new tag id

# Download dataset

In [13]:
'''
from Bio import Entrez
Entrez.email = "abolhasani.eliya@gmail.com"     
with Entrez.esearch(db='nucleotide', term="Arabidopsis thaliana") as handle:
    result = Entrez.read(handle)

print(result)
genome_ids = result['IdList']

for genome_id in genome_ids:
    print(genome_id)
    record = Entrez.efetch(db="nucleotide", id=genome_id, rettype="fasta", retmode="text")        
    with open(f'{genome_id}.fasta', 'w') as f:
        f.write(record.read())
    break
''';
'''
from Bio import Entrez
Entrez.email = "abolhasani.eliya@gmail.com"     
record = Entrez.efetch(db="nucleotide", id="NC_054143.4", rettype="fasta", retmode="text")        
with open(f'data.fasta', 'w') as f:
    f.write(record.read())
''';

In [14]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/439/995/GCA_000439995.3_AzaInd2.1/GCA_000439995.3_AzaInd2.1_genomic.fna.gz

--2021-11-14 18:10:04--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/439/995/GCA_000439995.3_AzaInd2.1/GCA_000439995.3_AzaInd2.1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.7, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85647577 (82M) [application/x-gzip]
Saving to: ‘GCA_000439995.3_AzaInd2.1_genomic.fna.gz’

CA_000439995.3_AzaI  27%[====>               ]  22.82M  7.26MB/s    eta 8s     ^C


In [263]:
!gzip -d ./GCA_000439995.3_AzaInd2.1_genomic.fna.gz

# Download data from Mirbase

In [28]:
directory = 'miRBase_driven_data'

In [18]:
base = "https://www.mirbase.org/ftp/CURRENT"        
!rm -r {directory}
!mkdir -p {directory}
!wget {base}/aliases.txt.gz -P ./{directory}/       ; gzip -d ./{directory}/aliases.txt.gz 
!wget {base}/hairpin.fa.gz -P ./{directory}/           ; gzip -d ./{directory}/hairpin.fa.gz 
!wget {base}/hairpin_high_conf.fa.gz -P ./{directory}/ ; gzip -d ./{directory}/hairpin_high_conf.fa.gz 
!wget {base}/mature.fa.gz -P ./{directory}/            ; gzip -d ./{directory}/mature.fa.gz 
!wget {base}/mature_high_conf.fa.gz -P ./{directory}/  ; gzip -d ./{directory}/mature_high_conf.fa.gz
!wget {base}/miRNA.str.gz -P ./{directory}/            ; gzip -d ./{directory}/miRNA.str.gz 
!wget {base}/miRNA.xls.gz -P ./{directory}/            ; gzip -d ./{directory}/miRNA.xls.gz 
!wget {base}/organisms.txt.gz -P ./{directory}/        ; gzip -d ./{directory}/organisms.txt.gz

--2021-11-14 19:08:31--  https://www.mirbase.org/ftp/CURRENT/aliases.txt.gz
Resolving www.mirbase.org (www.mirbase.org)... 130.88.97.249
Connecting to www.mirbase.org (www.mirbase.org)|130.88.97.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480536 (469K) [application/x-gzip]
Saving to: ‘./miRBase_driven_data/aliases.txt.gz’

aliases.txt.gz      100%[===================>] 469.27K   469KB/s    in 1.0s    

2021-11-14 19:08:32 (469 KB/s) - ‘./miRBase_driven_data/aliases.txt.gz’ saved [480536/480536]

--2021-11-14 19:08:33--  https://www.mirbase.org/ftp/CURRENT/hairpin.fa.gz
Resolving www.mirbase.org (www.mirbase.org)... 130.88.97.249
Connecting to www.mirbase.org (www.mirbase.org)|130.88.97.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1547350 (1.5M) [application/x-gzip]
Saving to: ‘./miRBase_driven_data/hairpin.fa.gz’

hairpin.fa.gz       100%[===================>]   1.48M   995KB/s    in 1.5s    

2021-11-14 19:08:34 (995 K

In [29]:
df = fasta_to_df(f'./{directory}/mature.fa')
#df = fasta_to_df('./Data/mature_high_conf.fa')
df['organism'] = df['tag'].apply(lambda x: x[:3])
print(df.shape)
df.head(2)

(48885, 3)


,tag,data,organism
0,cel-let-7-5p MIMAT0000001 Caenorhabditis elega...,UGAGGUAGUAGGUUGUAUAGUU,cel
1,cel-let-7-3p MIMAT0015091 Caenorhabditis elega...,CUAUGCAAUUUUCUACCUUACC,cel


In [30]:
organism = pd.read_csv(f'./{directory}/organisms.txt',sep='\t')
organism.columns = [c.replace('#','') for c in organism.columns] # remove sharp from columns
print(organism.shape)
organism.head(2)

(285, 5)


,organism,division,name,tree,NCBI-taxid
0,aqu,AQU,Amphimedon queenslandica,Metazoa;Porifera;,400682
1,nve,NVE,Nematostella vectensis,Metazoa;Cnidaria;,45351


In [31]:
items = list(organism['tree'].unique())
items.sort(key=len)
items

['Viruses;',
 'Mycetozoa;',
 'Alveolata;',
 'Metazoa;Porifera;',
 'Metazoa;Cnidaria;',
 'Viridiplantae;Chlorophyta;',
 'Viridiplantae;Embryophyta;',
 'Viridiplantae;Coniferophyta;',
 'Viridiplantae;Magnoliophyta;',
 'Metazoa;Bilateria;Deuterostoma;',
 'Chromalveolata;Heterokontophyta;',
 'Metazoa;Bilateria;Ecdysozoa;Nematoda;',
 'Metazoa;Bilateria;Lophotrochozoa;Annelida;',
 'Metazoa;Bilateria;Lophotrochozoa;Nemertea;',
 'Metazoa;Bilateria;Lophotrochozoa;Mollusca;',
 'Viridiplantae;Magnoliophyta;monocotyledons;',
 'Metazoa;Bilateria;Deuterostoma;Hemichordata;',
 'Metazoa;Bilateria;Deuterostoma;Echinodermata;',
 'Metazoa;Bilateria;Lophotrochozoa;Brachiopoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Hexapoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Crustacea;',
 'Metazoa;Bilateria;Lophotrochozoa;Platyhelminthes;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Chelicerata;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Mandibulata;',
 'Viridiplantae;Magnoliophyta;eudicotyledons;Poaceae;',
 'M

In [32]:
selectedTree = organism[organism['tree'].apply(lambda x: "Viridiplantae;" in x)]
print(selectedTree.shape)
selectedTree.head(5)

(86, 5)


,organism,division,name,tree,NCBI-taxid
66,cre,CRE,Chlamydomonas reinhardtii,Viridiplantae;Chlorophyta;,3055
67,pta,PTA,Pinus taeda,Viridiplantae;Coniferophyta;,3352
68,ppt,PPT,Physcomitrella patens,Viridiplantae;Embryophyta;,3218
69,smo,SMO,Selaginella moellendorffii,Viridiplantae;Embryophyta;,88036
70,ath,ATH,Arabidopsis thaliana,Viridiplantae;Magnoliophyta;eudicotyledons;Bra...,3702


In [33]:
selected = df[df['organism'].isin(selectedTree['organism'])]
print(selected.shape)
selected.head()

(10414, 3)


,tag,data,organism
316,ath-miR156a-5p MIMAT0000166 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath
317,ath-miR156a-3p MIMAT0031865 Arabidopsis thalia...,GCUCACUGCUCUUUCUGUCAGA,ath
318,ath-miR156b-5p MIMAT0000167 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath
319,ath-miR156b-3p MIMAT0031866 Arabidopsis thalia...,UGCUCACCUCUCUUUCUGUCAGU,ath
320,ath-miR156c-5p MIMAT0000168 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath


In [34]:
df_to_fasta(selected,'./Temp/mature_microRNA_queries.fasta')

# Remove redundant

## cdhit-est

In [41]:
!cdhit/cd-hit-est -i ./Temp/mature_microRNA_queries.fasta  -o ./Temp/NR_mature_microRNA_queries.fasta \
    -c 1 -r 0 -G 1 -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 \
    -AS 99999999 -s 0 -S 0 

Program: CD-HIT, V4.8.1 (+OpenMP), Oct 23 2021, 21:45:39
Command: cdhit/cd-hit-est -i
         ./Temp/mature_microRNA_queries.fasta -o
         ./Temp/NR_mature_microRNA_queries.fasta -c 1 -r 0 -G 1
         -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 -AS 99999999
         -s 0 -S 0

Started: Sun Nov 14 19:23:14 2021
                            Output                              
----------------------------------------------------------------
total seq: 10414
longest and shortest : 28 and 17
Total letters: 222978
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 1M
Buffer          : 1 X 12M = 12M
Table           : 1 X 16M = 16M
Miscellaneous   : 0M
Total           : 30M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 96149440

comparing sequences from          0  to      10414
..........    10000  finished       5817  clusters

    10414  finished       6028  clusters

Approximate

## reformat

In [42]:
with open('./Temp/NR_mature_microRNA_queries.fasta.clstr','r') as file:
    text = file.read()
lines = [line for line in text.split('\n') if len(line) > 0]
cluster = []
seqid = []
last_cluster = ""
for l in lines:
    if(l[0]=='>'):        
        last_cluster = l.replace('>Cluster ',"C")
    else:        
        cluster.append(last_cluster)
        seqid.append(l.split(', >')[1].split('...')[0])                
seq2cluster = pd.DataFrame({'seqid': seqid,'cluster': cluster})
print(seq2cluster.shape)
seq2cluster.head(2)    

(10414, 2)


,seqid,cluster
0,cst-miR11332,C0
1,stu-miR7994b-5p,C1


In [43]:
df = fasta_to_df("./Temp/mature_microRNA_queries.fasta")
df['accession'] = df['tag'].apply(lambda x : x.split(' ')[0])
seq2cluster = pd.merge(df,seq2cluster,how="inner",left_on='accession',right_on="seqid")[['cluster','seqid','tag']]
print(seq2cluster.shape)
display(seq2cluster.head(2))
seq2cluster.to_csv('./Temp/seq2cluster.csv',index=False)

(10414, 3)


,cluster,seqid,tag
0,C5495,ath-miR156a-5p,ath-miR156a-5p MIMAT0000166 Arabidopsis thalia...
1,C1199,ath-miR156a-3p,ath-miR156a-3p MIMAT0031865 Arabidopsis thalia...


In [44]:
# todo: sorted first by cluster then by seqid
seq2cluster.sort_values("cluster").head(2)

,cluster,seqid,tag
9422,C0,cst-miR11332,cst-miR11332 MIMAT0044622 Cucumis sativus miR1...
7002,C1,stu-miR7994b-5p,stu-miR7994b-5p MIMAT0031188 Solanum tuberosum...


In [45]:
df = fasta_to_df("./Temp/NR_mature_microRNA_queries.fasta")
df['tag'] = df['tag'].apply(lambda x : x.split(' ')[0])
df = pd.merge(df,seq2cluster,how="inner",left_on='tag',right_on="seqid")[['cluster','data']]

lines = []
df.apply(lambda row: lines.append(f">{row['cluster']}\n{row['data']}\n"),axis=1)
print(df.shape)
with open('./Temp/BLASTn_queries.fasta','w') as file:
    file.write(''.join(lines))

(6028, 2)


# BlastN

!sudo apt-get install ncbi-blast+


In [46]:
!makeblastdb -in input_genome.fna \
             -dbtype nucl \
             -out ./Temp/blastn_database



Building a new DB, current time: 11/14/2021 19:23:46
New DB name:   /home/jupyter/plant_microRNA_prediction/Temp/blastn_database
New DB title:  input_genome.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 126142 sequences in 9.84965 seconds.


In [17]:
header = 'qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'

In [18]:
!blastn -query ./Temp/BLASTn_queries.fasta \
        -out ./Temp/BLASTn_result \
        -num_threads {mp.cpu_count()} \
        -db ./Temp/blastn_database \
        -word_size 7 \
        -penalty -3 \
        -reward 2 \
        -gapopen 5 \
        -gapextend 2 \
        -outfmt '6 qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'       

In [19]:
df_blastn = pd.read_csv('./Temp/BLASTn_result', sep='\t',header=None)
df_blastn.columns = header.replace("  "," ").split(" ")
print(df_blastn.shape)
df_blastn.head(2)

(326849, 27)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,gaps,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen
0,C5495,AMWY02099822.1,1,20,1769,1750,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,0,100.0,1/-1,1,-1,minus,100,100,20,3308
1,C5495,AMWY02082313.1,1,20,5954,5973,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,0,100.0,1/1,1,1,plus,100,100,20,8471


In [20]:
threshold = 4
df_blastn['Nonconformity'] = df_blastn['qlen'] - (abs(df_blastn['qend'] - df_blastn['qstart']) + 1) + df_blastn['gaps'] + df_blastn['mismatch']
df_blastn = df_blastn[df_blastn['Nonconformity'] <= threshold]
print(df_blastn.shape)
df_blastn.head(2)

(80217, 28)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen,Nonconformity
0,C5495,AMWY02099822.1,1,20,1769,1750,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,100.0,1/-1,1,-1,minus,100,100,20,3308,0
1,C5495,AMWY02082313.1,1,20,5954,5973,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,100.0,1/1,1,1,plus,100,100,20,8471,0


In [21]:
# remore redundancy and hold best one base of Nonconformity value
df_blastn = df_blastn.sort_values(["Nonconformity", "evalue"], ascending = (True, True))
df_blastn = df_blastn.drop_duplicates(subset=['sseqid','sstart', 'send','sstrand'], keep='first')
df_blastn.to_csv('./Temp/filtered_out_blastn.csv')
print(df_blastn.shape)

(66445, 28)


# Result of the blastn to bed file

In [22]:
flanking_value = 200
df = df_blastn[['sseqid', 'sstart', 'send', 'sstrand','slen']]
df['ones'] = 1

<ipython-input-22-d52c72306552>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ones'] = 1


In [23]:
def switch(row):
    if(row['sstart'] > row['send']):        
        temp = row['sstart']
        row['sstart'] = row['send']
        row['send'] = temp
    return row
df = df.apply(lambda row: switch(row), axis=1)

In [24]:
def convert(inp):
    if(inp == "plus"):
        return "forward"
    if(inp == "minus"):
        return "reverse"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['strand'] = df['sstrand'].apply(lambda x: convert(x))

In [25]:
def convert2sign(inp):
    if(inp == "plus"):
        return "+"
    if(inp == "minus"):
        return "-"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['sign'] = df['sstrand'].apply(lambda x: convert2sign(x))

In [26]:
df['hit_length'] = df.apply(lambda row: abs(row['send'] - row['sstart']) + 1 ,axis=1)

## convert sstart and send from location to index (range)

In [27]:
df['sstart'] = df['sstart'].apply(lambda x: x - 1)

In [28]:
df['downstream_flanking'] = df['sstart'].apply(lambda x:  flanking_value if x > flanking_value else x)

In [29]:
df['upstream_flanking'] = df.apply(lambda row:  flanking_value if (row['send']+flanking_value) <= row['slen'] else row['slen'] - row['send'],axis=1)

In [30]:
df['hit_start'] = df.apply(lambda row: row['downstream_flanking'] if row['sign'] == "+" else row['upstream_flanking'],axis=1)

In [31]:
df['hit_end'] = df.apply(lambda row: row['downstream_flanking'] + row['hit_length'] if row['sign'] == "+" else row['upstream_flanking'] + row['hit_length'],axis=1)

In [32]:
df['sstart'] = df['sstart'].apply(lambda x: max(x - flanking_value, 0))
df['send'] = df.apply(lambda row: min(row['send'] + flanking_value , row['slen']),axis=1)

In [33]:
df['tag'] = df.apply(lambda row: f">{row['sseqid']}:{row['sstart']}-{row['send']}({row['sign']})",axis=1)
df['reformated_tag'] = df['tag'].apply(lambda t: reformat(t))
df[['tag', 'reformated_tag', 'hit_start', 'hit_end']].to_csv('./Temp/hit_index_info.csv')#, index=False)

In [34]:
df['location_tag'] = df.apply(lambda row: f">{row['sseqid']}|{row['sign']}|{row['sstart'] + 1}-{row['send']}|{row['hit_start']+1}-{row['hit_end']}",axis=1)
df[['location_tag']].to_csv('./Temp/pipe_seprated_location_list.csv',index=False)

In [35]:
df[['sseqid','sstart','send','strand','ones', 'sign']].to_csv('./Temp/extension_index.bed', 
        index=False, header=False, sep="\t")

# Extention


In [36]:
# !sudo apt-get install bedtools

In [37]:
!bedtools getfasta -fi ./input_genome.fna -fo ./Temp/extended_original.txt -s -bed ./Temp/extension_index.bed
!rm input_genome.fna.fai

index file ./input_genome.fna.fai not found, generating...


In [34]:
# todo: remove duplicated
'''
df = fasta_to_df("./Temp/extended.txt")
df = df.drop_duplicates(subset=['tag'], keep='first')
df_to_fasta(df,"./Temp/extended.txt")
len(df['tag'].unique())
''';

# Convert hit region to upper case and other region to lower case

In [38]:
ext = fasta_to_df('./Temp/extended_original.txt')
info = pd.read_csv('./Temp/hit_index_info.csv')
info['tag'] = info['tag'].apply(lambda x: x[1:])
print(info.shape)
info.head(2)

(66445, 5)


,Unnamed: 0,tag,reformated_tag,hit_start,hit_end
0,132836,AMWY02059828.1:2832-3256(+),>AMWY020598281_2832-3256_+_,200,224
1,300170,AMWY02004761.1:1853-2277(+),>AMWY020047611_1853-2277_+_,200,224


In [39]:
ext = ext.sort_values(by=['tag']).reset_index()
ext['help_tag'] = ext.apply(lambda r: r['tag']+ str(r.name),axis=1)
del ext['tag']

info = info.sort_values(by=['tag']).reset_index()
info['help_tag'] = info.apply(lambda row: row['tag']+ str(row.name),axis=1)
def redefined_tag(row):
    tag = row['tag']
    [sstart, send] = tag.split(':')[-1].split('(')[0].split('-')
    sstart = int(sstart) + 1
    sign = tag.split('(')[-1].split(')')[0]    
    return f"{tag.split(':')[0]}|{sstart}-{send}|{row['hit_start']+1}-{row['hit_end']}|{sign}"
info['tag'] = info.apply(lambda row: redefined_tag(row),axis=1)
ext = pd.merge(ext,info,how='inner', on='help_tag')

def emphasis_hit(row):
    seq = list(row['data'].lower())            
    s = row['hit_start']
    e = row['hit_end']
    seq[s:e] = list(''.join(seq[s:e]).upper())    
    return ''.join(seq)
    
ext['data'] = ext.apply(lambda row: emphasis_hit(row),axis=1)
df_to_fasta(ext[['tag','data']],"./Temp/extended_modified.txt")

# Extended validation

In [ ]:
df_blastn['hit'] = df_blastn['sseq'].apply(lambda x: x.replace('-', ''))
info = pd.read_csv('./Temp/hit_index_info.csv')
ext = fasta_to_df('./Temp/extended2.txt')

counter = 0
for index in df_blastn.index:
    hit = df_blastn['hit'][index]
    row = info[info['Unnamed: 0']== index].reset_index()
    tag = row['tag'][0][1:]
    hs = row['hit_start'][0]
    he = row['hit_end'][0]        
    tag = tag.replace('(+)',f"|{hs}-{he}(+)")
    tag = tag.replace('(-)',f"|{hs}-{he}(-)")            
    seq = ext[ext['tag']==tag]['data'].iloc[0]
    seq = ext[ext['tag']==tag]['data'].iloc[0]    
    if(seq[hs:he] != hit):
        print(tag, df_blastn['slen'][index])
        print(seq[hs:he])
        print(hit)
        print('\n\n')
        counter += 1                

In [178]:
df = pd.read_csv('check1.csv')
df.columns = ['tag', 'data']
def do(x):
    x = x[1:]
    x = x.split('|')
    s = int(x[2].split('-')[0]) - 1
    e = x[2].split('-')[1]
    return f"{x[0]}:{s}-{e}({x[1]})"
df['tag'] = df['tag'].apply(lambda x: do(x))
df.head(2)

,tag,data
0,AMWY02000003.1:4084-4506(+),ATACAATTGTCACATagtttacattattaatttccgcttaatttat...
1,AMWY02000003.1:4391-4810(+),ttctaattgcaaatttatgttatatttttaaatagaaaggGAGATT...


In [176]:
for t in df['tag']:
    check_seq = df[df['tag']==t]['data'].iloc[0]
    ext_seq = ext[ext['tag']==t]['data'].iloc[0]
    if(check_seq != ext_seq):
        print('error')        

In [167]:
ext = fasta_to_df('./Temp/extended.txt')
ext.head(2)

,tag,data
0,AMWY02059828.1:2832-3256(+),AAAGAATCAGCAATGGAAAAATAACCGGTTCTTAATTCAGcataac...
1,AMWY02004761.1:1853-2277(+),actaataatgCATGGCCATATATATCAAATCTACCATATgccattt...


# RNA 2d prediction

## Mfold

In [14]:
'''
# installation
!wget http://www.unafold.org/download/mfold-3.6.tar.gz
!tar -xvf ./mfold-3.6.tar.gz; rm ./mfold-3.6.tar.gz
%cd ./mfold-3.6
!./configure
!make
!make install
%cd ..
!sudo apt install texlive-font-utils
''';

In [15]:
#todo : add all hyperparameter(options) to GUI

In [17]:
counter = 0
base = "./secondary_structure/mfold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df('./Temp/extended_modified.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/SEQ.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    if(counter >= 200):
        break

In [18]:
remove_lock = False
def run_mfold(tag):
    tag = reformat(tag)
    %cd {base + tag}
    !mfold  SEQ="SEQ.FASTA" T=20     
    if(not remove_lock):
        !find . -not -name "*.ct" -not -name "*.pdf" -not -name "*SEQ.FASTA" -not -type d -delete
    %cd ../../..

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count())  
    pool.map(run_mfold, df['tag'].iloc[:200])

/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000031|4085-4506|201-222|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000261|484-903|201-220|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000391|321-573|201-218|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000141|194-614|201-221|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000471|1432-1851|201-220|-/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000511|4616-5029|201-214|-/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000531|455-874|201-220|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000231|268-683|201-216|-/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000311|5367-5786|201-220|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000881|2344-2731|201-219|+/home/jupyter/plant_

190,7	9	19	6	
Structure plots generated.
All done.
3	19	REUSE= NO
19	19	19	4	200,210,19	2	8	2	220,230,SEQ.pnt created.
Sequence length is 419
3	240,5	RNA free energy files (version 2.3) at 20 degrees created.
250,260,/home/jupyter/plant_microRNA_prediction
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000191|1-291|201-219|-10,20,30,40,50,60,
270,280,290,300, 
1	70,80,90,1	9	End of Fill
Save file created using nafold.
Minimum folding energy is -81.90 kcal/mol.
4	
Structure plots generated.
All done.
mfold version 3.6
3	1	100,110,7	Energy increment is 4.09 kcal/mol.
1	REUSE= NO
1	8	1	120,130,1	5	H-num file created from plot file.
SEQ.pnt created.
1	9	3	1	140,150,3	Sequence length is 291
1,2,3,4,5,6,7,4	160,170,/home/jupyter/plant_microRNA_prediction8,9,10,11,12,
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000911|11460-11868|201-219|+180,RNA free energy files (version 2.3) at 20 degrees created.
10,20,
6	13,14, 
190,30,40,50,60,2	mfo


11	14	/home/jupyter/plant_microRNA_prediction110,13	110,
170,180,/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020001191|1555-1973|201-219|-mfold version 3.6

mfold version 3.6
11	120,120,RNA free energy files (version 2.3) at 20 degrees created.
10,REUSE= NO
H-num file created from plot file.
3	7	190,130,140,mfold version 3.6
REUSE= NO
130,140,1,2,3,4,5,6,20,30,40,50,4	200,/home/jupyter/plant_microRNA_prediction
13	REUSE= NO
9	
150,/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000511|6024-6442|201-219|-SEQ.pnt created.
Sequence length is 419
150,7,8,9,10,11,60,70,80,12	SEQ.pnt created.
Sequence length is 419
210,160,SEQ.pnt created.
Sequence length is 419
160,170,RNA free energy files (version 2.3) at 20 degrees created.
12, 
RNA free energy files (version 2.3) at 20 degrees created.
90,100,15	2	170,180,mfold version 3.6
10,20,30,40,50,13	180,220,24	RNA free energy files (version 2.3) at 20 degrees created.
10,110,120,130,10,20,30,40

2	/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000301|687-1105|201-219|-140,150,270,280,
2	Suboptimal foldings created.
6	160,290,300,mfold version 3.6
Energy dot plot created.
10	170,180,24	11	12	REUSE= NO
23	310,320,3	11	190,9	5	SEQ.pnt created.
330,340,350,360,1	2	12	200,3	Sequence length is 419

Structure plots generated.
All done.
370,380,390,400,15	210,220,5	RNA free energy files (version 2.3) at 20 degrees created.
16	410, 
End of Fill
Save file created using nafold.
230,2	Minimum folding energy is -145.10 kcal/mol.
Energy increment is 7.25 kcal/mol.
10,20,30,40,50,3	
Structure plots generated.
All done.
3	240,H-num file created from plot file.
60,11	250,1,70,80,90,/home/jupyter/plant_microRNA_prediction
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020001541|671-1094|201-224|+4	
2,3,4,5,6,260,12	100,24	13	25	12	
Structure plots generated.
All done.
7,8,9,6	270,110,3	mfold version 3.6
4	10,11,12,/home/jupyter/plant_microRNA_pre

290,50,60,70,24	60,70,80,REUSE= NO
90,100,330,10	300,310,80,90,90,100,SEQ.pnt created.
340,350,110,9	SEQ.pnt created.
Sequence length is 215
110,100,110,320,330,13	5	8	Sequence length is 420
360,370,380,120,130,140,16	RNA free energy files (version 2.3) at 20 degrees created.
10,20,30,120,130,4	120,130,140,340,350,15	RNA free energy files (version 2.3) at 20 degrees created.
10,20,30,390,400,410,420, 
End of Fill
22	40,50,60,70,150,160,140,150,160,8	360,370,150,160,4	40,50,60,70,Save file created using nafold.
Minimum folding energy is -135.00 kcal/mol.
Energy increment is 6.75 kcal/mol.
170,180,6	80,90,100,170,180,380,390,400,170,180,4	25	22	11	80,90,190,110,120,190,200,Suboptimal foldings created.
Energy dot plot created.
190,200,H-num file created from plot file.
410, 
End of Fill
Save file created using nafold.
11	100,110,200,130,140,210,220,10	
Structure plots generated.
All done.
210,Minimum folding energy is -103.20 kcal/mol.
Energy increment is 5.16 kcal/mol.
120,130,230,240,25

...

13	3	5	14	280,11	1,7,8,9,4	REUSE= NO
290,300,2,3,4,5,6,6	10, 
14	310,SEQ.pnt created.
Sequence length is 263
5	7,8,11	13	320,330,340,350,3	16	RNA free energy files (version 2.3) at 20 degrees created.
9,10,11,12,13,15	360,21	10,20,30,8	14,15, 
Suboptimal foldings created.
Energy dot plot created.
10	22	Suboptimal foldings created.
Energy dot plot created.
9	370,380,390,15	6	10	40,50,60,400,410,420, 
End of Fill
Save file created using nafold.
Minimum folding energy is -134.20 kcal/mol.
Energy increment is 6.71 kcal/mol.
14	4	70,80,Suboptimal foldings created.
Energy dot plot created.
5	12	15	7	H-num file created from plot file.
90,100,14	10	17	6	110,120,12	1,2,3,4,5,4	Suboptimal foldings created.
9	6,7,8,9,10,130,140,Energy dot plot created.
10	16	1	11	
Structure plots generated.
All done.16	150,160,23	
22	11, 
7	170,180,1	5	190,200,16	6	8	11	15	210,220,13	18	7	230,240,250,260, 
End of Fill
Save file created using nafold.
13	5	
Structure plots generated.
All done.
/home/jupyter/plant_m

SEQ.pnt created.
All done.

Structure plots generated.
5	150,160,4	10	150,160,6	10	11	Sequence length is 281
170,5	8	170,24	RNA free energy files (version 2.3) at 20 degrees created.
10,180,190,
Structure plots generated.
All done.
180,200,Suboptimal foldings created.
Energy dot plot created.
10	20,30,40,50,13	8	27	190,22	210,/home/jupyter/plant_microRNA_prediction60,70,29	
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020003031|4770-5187|201-218|+200,
/home/jupyter/plant_microRNA_prediction11	
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002411|5665-6079|201-215|+220,80,
11	210,220,230,90,100,/home/jupyter/plant_microRNA_prediction
mfold version 3.6
22	/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020003061|8575-8993|201-219|-
mfold version 3.6
24	11	230,5	240,6	110,REUSE= NO
REUSE= NO
1	mfold version 3.6
240,6	250,9	SEQ.pnt created.
Sequence length is 4187	120,25	
250,260,28	130,23	REUSE= NO
SEQ.pnt created.


10	14	3	180,/home/jupyter/plant_microRNA_prediction
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020003291|25-445|201-221|-
14	190,H-num file created from plot file.
200,1,2,3,4,5,mfold version 3.6
REUSE= NO
9	8	2	210,6,7,8,9,10,11,12,SEQ.pnt created.
Sequence length is 421
4	14	220,13,14,15,16,17,RNA free energy files (version 2.3) at 20 degrees created.
6	17	230,18,19,20,21,10,20,30,11	23	17	240,22,23,24, 
40,50,60,70,4	19	4	250,260,11	80,90,100,1	270,280,15	110,290,
Structure plots generated.
All done.
120,130,300,9	3	140,150,5	310,320,330,15	160,340,350,360,7	18	170,12	370,380,390,400,410, 
End of Fill
1	24	180,5	18	/home/jupyter/plant_microRNA_prediction
Save file created using nafold.
Minimum folding energy is -114.20 kcal/mol.
Energy increment is 5.71 kcal/mol.
5	190,12	2	16	200,210,Suboptimal foldings created.
H-num file created from plot file.
Energy dot plot created.
10	6	220,1	1,2,3,4,
Structure plots generated.
All done.
4	230,240,5,6,7,8,9,10,11,12

5,6,7,8,9,10,230,20	Suboptimal foldings created.
Energy dot plot created.
90,100,110,12	11,12,13,14,15,16,6	240,10	120,17,18,19, 
250,130,140,22	260,150,17	/home/jupyter/plant_microRNA_prediction270,280,
160,3	12	170,25	1	11	290,300,25	180,2	310,190,
Structure plots generated.
All done.
11	1	320,330,340,200,23	350,360,370,210,380,390,400,410,420, 
End of Fill
Save file created using nafold.
Suboptimal foldings created.
Energy dot plot created.
10	220,13	Minimum folding energy is -155.80 kcal/mol.
Energy increment is 7.79 kcal/mol.
18	230,4	12	H-num file created from plot file.
26	20	240,/home/jupyter/plant_microRNA_prediction
2	1,2,3,4,5,250,260,12	3	2	6,7,8,9,10,270,24	11,12,13, 
280,290,300,310,11	320,19	27	21	330,340,13	14	350,360,5	370,380,390,400,410, 
End of Fill
13	3	Save file created using nafold.
Minimum folding energy is -159.00 kcal/mol.
Energy increment is 7.95 kcal/mol.
4	3	Suboptimal foldings created.
Energy dot plot created.
10	2	H-num file created from plot file.
12	22	

15	H-num file created from plot file.

Structure plots generated.
All done.
60,70,80,H-num file created from plot file.
1,2,11	1,2,3,4,90,100,3,4,5,6,7,8,9,5,6,7,8,9,10,14	110,120,10,11, 
11	11,12,13,14,15,16,130,140,3	17,18,19,20,150,160,/home/jupyter/plant_microRNA_prediction
21, 
170,13	16	2	24	180,12	Suboptimal foldings created.
Energy dot plot created.
10	190,15	200,210,220,12	4	230,3	14	17	240,2	11	250,Suboptimal foldings created.
Energy dot plot created.
10	260,270,13	13	280,16	290,300,18	310,320,5	15	330,340,4	3	350,360,370,1	14	380,390,400,410, 
End of Fill
Save file created using nafold.
Minimum folding energy is -143.70 kcal/mol.
Energy increment is 7.18 kcal/mol.
11	1	14	H-num file created from plot file.
16	19	1,2,3,4,5,6	6,7,8,9,10,5	4	11,12,13, 
2	15	2	15	12	1	17	6	7	5	Suboptimal foldings created.
Energy dot plot created.
10	3	16	3	18	16	13	20	7	8	6	11	4	17	17	4	21	19	8	9	14	5	12	7	18	18	22	9	5	1	15	
Structure plots generated.
All done.
8	13	6	23	19	19	6	
Structure plots

In [61]:
'''
base = "secondary_structure/mfold/"
for directory in glob.glob(f"{base}*"):    
    tag = directory[len(base):]
    ct_files = glob.glob(f'{directory}/*.ct')        
    try:
        ct_files.remove(f'{base}{tag}/SEQ.ct')
    except:
        print(directory)
        print(ct_files)
        print("*****************")
    for file in ct_files:        
        shutil.copy(file, './1.ct')
        #dot = ct2dot_bracket('./1.ct')
        #dot = dot.split('\n')
        #with open('./2.ct', 'w') as stream:
            #stream.write(bracket_to_ct(tag, dot[0] , dot[1] , "(0)"))        
        #ct1 = '\n'.join(reformatCT('./1.ct').split('\n')[1:])
        #ct2 = '\n'.join(reformatCT('./2.ct').split('\n')[1:])
        #if(ct1 != ct2):
            #print(file)
        ct = reformatCT('./1.ct')
        [nucleotide, index, values] = get_ct_data(ct)        
        #print(is_nested( index,  values))
        if(not is_nested( index,  values)):
            print("************")             
'''

## Mxfold2

In [57]:
#!wget https://github.com/keio-bioinformatics/mxfold2/releases/download/v0.1.1/mxfold2-0.1.1.tar.gz
#!pip3 install mxfold2-0.1.1.tar.gz
#!rm mxfold2-0.1.1.tar.gz

In [58]:
!mxfold2 predict ./extended.txt > secondary_structure/mxfold2_result.txt

>AMWY02059828.1:2832-3256(+)
AAAGAATCAGCAATGGAAAAATAACCGGTTCTTAATTCAGcataacaaattattcaattataatatagcTGTAAAAGAAATCTAAGTCTATTTGATATAGATCGGAATTTacgcaaattaaaaatttccaaataaGCAGTTCCGACCTGAGATCTGAACCGAAAACGCAAGATCCATCTAAACTCTCACCTCGGTCTCCGATTCAGTTGATGCAAGGCGGGATCCAATTCGCCTTTTCATTCAATTACATTCACCAATAACAGCTCGCCATCtggcttttaataaaaagttgcCAATCGGTTCCCGACCTGCACCAAGCGAATTAGAGACCGCCGGTAACTGAATCATTCTACATTAATCCCCGACTCCTCCTTTTACACATAGCAACTTCGCCCAAGAagactaaaaagaaaaggaagctAAC
........................((((...........((..........................))..........................................................................................................................(((...(((...............((((..........)))).................................(((....))).....................)))...)))..........................))))........................................................................................ (35.0)
>AMWY02004761.1:1853-2277(+)
actaataatgCATGGCCATATATATCAAATCTACCATATgccatttaataattttccttttttcttcttctttctttttctctct

>AMWY02002487.1:2516-2939(+)
tctctcttctttttcttccttaacTTTTCATCCAGCTTCAACCTCCATTTAGATCaaagttattgaatttttttttcatcttatttatgtaaatatatattgtttccTGCGGAAACGAATCCATGAACAACAGTCAATCAGTCATTGTTTGCTGATGCAGCGTCATCAAGATTCGCATGCTGATGGGTCGAGCAAAGCAGTGAGAATCTTGATGATGCTGCATCGGCCATAATTGACTATAtctcgtcatcatcatcatcatcatccagtTTCAACCTCCATGTAAATCaagttattgaattatttggtAAATAGATACTGATTCCCGCAGAATTGAATCAATGAACAACAGTCAATCAGTCATTGTTTGCTGATGCAGCATCATCAAGATTCACATGCGAATGGGTCGAACCAAAGCAGTGA
............................................................................................................((((....(....)............................((....)).(((...........))).........((.((.((...((..................)).))..)).)).................................................................................................))))........................................((....))....................((....((......))...))..... (30.2)
>AMWY02089812.1:0-419(-)
TCtgtctatatttattttcttctcattcACTGTAGTAATTTAAGCCTATACAGTTCTGAGTTGACCNatttctttatataaagtTNATTTC

>AMWY02001968.1:1243-1665(+)
CTACCTAAACTCCATGCATGGCTCGTGCTAGCTTTctggtttcttcttttttctttaagggcttattataaatttgcaGCAAGCCTAAACCCTTCTTAATTTCAAGATCTCTCTTcatttgattctttctttctttttctagggattcttcttcttcttcttcttcttgtttgctGCTGGTGTATGTTGGTTTGAGAGATTGAAGCTGCCAGCATGATCTGGTAATATGGAAcctaatattatacatatacatatctatatctatatatatagatagatagttttagatttactctttaattatattaattcctCCACCTATAGTTTTAGAGttactctttaattatattaattcttccACCTCTTTGTATAGATAGATAGAAACATATAGAAGGTCTTAGATTTCCTTTGCTTTTAGATCC
...((....(.((..((...((....))...............................(((....................)))..........................................................................................)).)).)......))........................................................................................................................................................................................................................................ (27.0)
>AMWY02039981.1:1214-1636(-)
atttatgttttctattttataattaaaaaataaaaaaaaaataataaggtaatctctctctctttttcttttatattcatatgaGGTCT

In [143]:
df = fasta_to_df('secondary_structure/mxfold2_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
df.head(2)

,tag,data,bracket
0,AMWY02059828.1:2832-3256(+),AAAGAATCAGCAATGGAAAAATAACCGGTTCTTAATTCAGcataac...,........................((((...........((........
1,AMWY02004761.1:1853-2277(+),actaataatgCATGGCCATATATATCAAATCTACCATATgccattt...,.................................................


In [145]:
base = "./secondary_structure/mxfold2/"
!rm -r {base}
!mkdir -p {base}
for index, row in df.iterrows():    
    if(not os.path.exists(base + reformat(row['tag']))):
        os.makedirs(base + reformat(row['tag']))        
    tag = reformat(row['tag'])
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG)
        file.write(ct)    

## SPOT-RNA

In [8]:
#!git clone https://github.com/jaswindersingh2/SPOT-RNA.git
#%cd SPOT-RNA
#!wget 'https://www.dropbox.com/s/dsrcf460nbjqpxa/SPOT-RNA-models.tar.gz' || wget -O SPOT-RNA-models.tar.gz 'https://app.nihaocloud.com/f/fbf3315a91d542c0bdc2/?dl=1'
#!tar -xvzf SPOT-RNA-models.tar.gz && rm SPOT-RNA-models.tar.gz
#!sudo apt-get install python3.6
#!python3.6 -m pip install tensorflow==1.14.0 # or for gpu: tensorflow-gpu==1.14.0
#! python3.6 -m pip install -r requirements.txt

In [23]:
base = "./secondary_structure/spot_rna/"
!rm -r {base}
!mkdir -p {base}

In [29]:
!python3.6 ./SPOT-RNA/SPOT-RNA.py  --inputs ./extended.txt  --outputs '{base}'  --cpu 32 --plots True

>> Opening FASTA file...
>> Converting FASTA file from multiline to single line and writing to file.
>> Done!

Preparing tfr records file for SPOT-RNA:
100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.21s/it]

Predicting for SPOT-RNA model 0
100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.23s/it]

Predicting for SPOT-RNA model 1
100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.51s/it]

Predicting for SPOT-RNA model 2
100%|█████████████████████████████████████████████| 1/1 [00:13<00:00, 13.10s/it]

Predicting for SPOT-RNA model 3
100%|█████████████████████████████████████████████| 1/1 [00:16<00:00, 16.44s/it]

Predicting for SPOT-RNA model 4
100%|█████████████████████████████████████████████| 1/1 [00:18<00:00, 18.18s/it]

Post Processing and Saving Output

Finished!

Processsing Time 203.32813096046448 seconds


In [48]:
!rm {base}/*.bpseq
!rm {base}/*.prob
for file in glob.glob(f"{base}*.ct"):    
    f = file[len(base):-3] # .ct        
    f = reformat(f)        
    if(not os.path.exists(base + f)):
        os.makedirs(base + f)  
    header = reformatCT(file).split("\n")[0]    
    with open(f"{base}{f}.dot", 'w') as stream:        
        stream.write(ct2dot_bracket(file))
    !RNAeval "{base}{f}.dot" -T 20 -v 
    #shutil.move(file, f"{base}{f}/{f}.ct")    

rm: cannot remove './RNA_secondary_structure/spot_rna//*.bpseq': No such file or directory
rm: cannot remove './RNA_secondary_structure/spot_rna//*.prob': No such file or directory
External loop                           :  -371
Interior loop (  3, 32) AU; (  4, 31) GC:  -256
Interior loop (  4, 31) GC; (  5, 30) AU:  -294
Interior loop (  5, 30) AU; (  6, 29) AU:  -122
Hairpin  loop (  6, 29) AU              :   716
Interior loop ( 38, 71) CG; ( 39, 70) AU:  -255
Interior loop ( 39, 70) AU; ( 40, 69) GC:  -256
Interior loop ( 40, 69) GC; ( 41, 68) CG:  -403
Interior loop ( 41, 68) CG; ( 49, 64) AU:   618
Interior loop ( 49, 64) AU; ( 50, 63) UA:  -155
Interior loop ( 50, 63) UA; ( 51, 62) UA:  -122
Interior loop ( 51, 62) UA; ( 52, 61) AU:  -165
Hairpin  loop ( 52, 61) AU              :   502
Interior loop ( 87,101) UA; ( 88,100) CG:  -294
Interior loop ( 88,100) CG; ( 89, 99) UA:  -256
Interior loop ( 89, 99) UA; ( 90, 98) AU:  -165
Hairpin  loop ( 90, 98) AU              :   573
Int

In [68]:
df = fasta_to_df('./secondary_structure/spot_rna/AMWY02059828.1:2832-3256(+).dot')
df = df.apply(lambda row: bracket_row(row) , axis=1)
bracket = df['bracket'][0].split(' ')[0]
ct = bracket_to_ct(df['tag'][0], df['data'][0], bracket, "(0)")
print(ct)

   424 dG =       0.0 AMWY02059828.1:2832-3256(+)
     1 A      0      2      0       1
     2 A      1      3      0       2
     3 A      2      4     32       3
     4 G      3      5     31       4
     5 A      4      6     30       5
     6 A      5      7     29       6
     7 U      6      8      0       7
     8 C      7      9      0       8
     9 A      8     10      0       9
    10 G      9     11      0      10
    11 C     10     12      0      11
    12 A     11     13      0      12
    13 A     12     14      0      13
    14 U     13     15      0      14
    15 G     14     16      0      15
    16 G     15     17      0      16
    17 A     16     18      0      17
    18 A     17     19      0      18
    19 A     18     20      0      19
    20 A     19     21      0      20
    21 A     20     22      0      21
    22 U     21     23      0      22
    23 A     22     24      0      23
    24 A     23     25      0      24
    25 C     24     26      0      25


## Vienna package

In [92]:
#!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_20_04/viennarna_2.4.18-1_amd64.deb -O viennarna.deb
#!sudo dpkg -i ./viennarna.deb
#!sudo apt-get -f install
#!rm viennarna.deb

In [131]:
base = "./secondary_structure/viennarna/"
!rm -r {base}
!rm ./secondary_structure/viennarna_result.txt
!mkdir -p {base}

rm: cannot remove './secondary_structure/viennarna_result.txt': No such file or directory


In [132]:
%cd {base}
!RNAfold --jobs=0 --infile ../../Temp/extended_modified.txt  --noPS -T 20 > ../viennarna_result.txt
%cd ../../

/home/jupyter/plant_microRNA_prediction/secondary_structure/viennarna
/home/jupyter/plant_microRNA_prediction


In [95]:
df = fasta_to_df('secondary_structure/viennarna_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
print(df.shape)
df.head(2)

(21225, 3)


,tag,data,bracket
0,AMWY02000003.1:4084-4506|200-222(+),auacaauugucacauaguuuacauuauuaauuuccgcuuaauuuau...,....((((((...))))))................((((((...((...
1,AMWY02000003.1:4391-4810|200-219(+),uucuaauugcaaauuuauguuauauuuuuaaauagaaagggagauu...,.....((.(((......))).))......(((((((.(((((((.....


In [125]:
for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)      
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG, False)
        file.write(ct)    

In [126]:
import glob
for file in glob.glob(f"{base}*.ps"):    
    f = file[len(base):-6] # _ss.ps 
    f = reformat(f)        
    shutil.move(file, f"{base}{f}/{f}.ps")    

## ContraFold

In [39]:
#!wget http://contra.stanford.edu/contrafold/contrafold_v2_02.tar.gz
#!tar -xvzf contrafold_v2_02.tar.gz && rm contrafold_v2_02.tar.gz
#%cd contrafold/src
#!make clean
#!make 
# to file must changed to be complieable # utility.hpp and optimization.c++ files

In [42]:
counter = 0
base = "./secondary_structure/contrafold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df('./Temp/extended.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/{tag}.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    if(counter >= 10):
        break

In [43]:
def run_contrafold(tag):
    tag = reformat(tag)    
    %cd contrafold/src
    !./contrafold predict ../..{base[1:]}{tag}/{tag}.FASTA > ../..{base[1:]}{tag}/{tag}.dot
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'r') as file:
        text = file.read()
    text = [l for l in text.split("\n") if l[:len(">structure")] != ">structure"]    
    header = text[0]
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write('\n'.join(text[1:]))    
    !RNAeval  ../..{base[1:]}{tag}/{tag}.dot -T 20 > ../..{base[1:]}{tag}/{tag}.dotdg    
    with open(f"../..{base[1:]}{tag}/{tag}.dotdg", 'r') as file:
        text = file.read()
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write(header + "\n" + text)    
    
    df = fasta_to_df(f'../..{base[1:]}{tag}/{tag}.dot')
    df = df.apply(lambda row: bracket_row(row) , axis=1)        
    tag = reformat(df['tag'][0])
    with open(f'../..{base[1:]}{tag}/{tag}.ct','w') as file:
        bracket = df['bracket'][0].split(' ')[0]        
        deltaG = df['bracket'][0].split(' ')[1]
        ct = bracket_to_ct(df['tag'][0], df['data'][0], bracket, deltaG, False)
        file.write(ct)    
    #!rm ../..{base[1:]}{tag}/{tag}.dot
    #!rm ../..{base[1:]}{tag}/{tag}.dotdg
    !rm ../..{base[1:]}{tag}/{tag}.FASTA
    %cd ../../        

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count() - 1)  
    pool.map(run_contrafold, df['tag'].iloc[:10])

In [56]:
s = 'CUCCCCUUGUCUACCAUCCCCAACUAGCGAGAGAGACAUUACCUACCUGAAUAGAAGAUCUCUCUCGAGCUCUCGagcucucucuuuuucuauaUCUCUGUCUCUUUGUGUCUCUGGAGCUUGUACUAACAUUAAUAUCGUGCACCAGCAGCAGUUGAAGCUGCCAGCAUGAUCUAAACUUCCUUCUCUGUAAAGGAUAGAUCGGAUCAUGUGGUAGCUUCACCUGUUGAUGGGAUCACGAAAGCGCCCCUCUUACUACUCUACAUUAAUUCUUUCUCGUUAUACAACCUCCCAGUAAGCAUGCUUUCAAAACCAACUUGAGuaaguuaauuuguuuagcuuuuguuuuuggcucuuccuuuacuuuaaauuuucucaucuggguuuuuguuauauauauguacuguuuuauauauguauuccu'
d = '............................((((((((..(...(((......))).)..))))))))(((((....)))))...................................((((.((((...(((.......(((((..(((.((((((.((((((((((.(((((((((.(.(((((((.......))))).)).).))))))))))))))))))).)))))).)))...))))).....................................)))..)))).))))....((((()))))..((((((....((((.(((((((.....)))(((.........)))................)))).))))....))))))....(((((((((((......)))))))))))....'
print(s[300],s[301])
print(d[300],d[301])

A U
( )


In [55]:
'''path = 'secondary_structure/contrafold/AMWY020333941_469-893_-_/AMWY020333941_469-893_-_.dot'
!RNAeval  {path} -T 20 -v'''; 

External loop                           :  -364
Interior loop ( 29, 66) GC; ( 30, 65) AU:  -294
Interior loop ( 30, 65) AU; ( 31, 64) GC:  -256
Interior loop ( 31, 64) GC; ( 32, 63) AU:  -294
Interior loop ( 32, 63) AU; ( 33, 62) GC:  -256
Interior loop ( 33, 62) GC; ( 34, 61) AU:  -294
Interior loop ( 34, 61) AU; ( 35, 60) GC:  -256
Interior loop ( 35, 60) GC; ( 36, 59) AU:  -294
Interior loop ( 36, 59) AU; ( 39, 56) UA:   103
Interior loop ( 39, 56) UA; ( 43, 54) CG:   305
Interior loop ( 43, 54) CG; ( 44, 53) UA:  -256
Interior loop ( 44, 53) UA; ( 45, 52) AU:  -165
Hairpin  loop ( 45, 52) AU              :   493
Interior loop ( 67, 80) GC; ( 68, 79) AU:  -294
Interior loop ( 68, 79) AU; ( 69, 78) GC:  -256
Interior loop ( 69, 78) GC; ( 70, 77) CG:  -403
Interior loop ( 70, 77) CG; ( 71, 76) UA:  -256
Hairpin  loop ( 71, 76) UA              :   533
Interior loop (116,292) GC; (117,291) GC:  -385
Interior loop (117,291) GC; (118,290) AU:  -294
Interior loop (118,290) AU; (119,289) GC

# Visualization

In [25]:
#https://github.com/ViennaRNA/forna
#http://varna.lri.fr/

# CT Analizer

In [125]:
# only select those not ran before
base = "./secondary_structure/mfold/"
df = fasta_to_df('./Temp/extended_modified.txt')

index_list =[]
for index, row in df.iterrows():    
    tag = reformat(row['tag'])    
    if(len(glob.glob(f'{base + tag}/*.ct')) != 0):
        index_list.append(index)
df = df.iloc[index_list,:]
print(df.shape)

(200, 2)


In [256]:
def get_tag_info(tag):
    data = tag.split('|')
    hit_start = int(data[2].split('-')[0]) - 1 
    hit_end = int(data[2].split('-')[1])
    sign = data[3]
    return [hit_start, hit_end, sign]

In [257]:
def get_deltaG(ct):
    ct_head = ct.split('\n')[0]
    if("dG = " in ct_head):    
        dG_patter = "dG = " 
    elif("dG= " in ct_head):    
        dG_patter = "dG= "
    elif("dG=" in ct_head):    
        dG_patter = "dG="
    elif("dG =" in ct_head):    
        dG_patter = "dG ="
    else:
        print('there is no dG')
    return float(ct_head.split(dG_patter)[-1].split(' ')[0])

In [258]:
def get_complementarity_in_hit_region(inc_srange, hit_len):    
    if(sum(inc_srange == 0) == hit_len):
        return "no"
    elif(sum(inc_srange != 0) == hit_len):
        return "fully_connected"
    else:    
        return "yes"    

In [259]:
def get_hit_self_complementarity(hit_start, hit_end, inc_srange):    
    if(((inc_srange <= hit_start) | (inc_srange > hit_end)).all()):
        return "no"
    return "yes"

In [260]:
def get_istar_min_max(inc_srange, hit_self_complementarity):
    nonzero_data_srange  = inc_srange[inc_srange!=0]
    if(hit_self_complementarity == 'yes'):
        return [np.nan, np.nan]
    return [nonzero_data_srange.min(), nonzero_data_srange.max()]

In [261]:
def get_continuous_pairing(hit_start, hit_end, istar_min, istar_max, hit_self_complementarity):    
    if(hit_self_complementarity == 'yes'):
        return "undifined"
    if(hit_end < istar_max and (hit_start+1) > istar_min):
        return "no"    
    return  "yes"

In [299]:
def get_mir_type(hit_start, hit_end, istar_min, istar_max, continuous_pairing, complementarity_in_hit_region, hit_self_complementarity):        
    if(continuous_pairing == "yes" and complementarity_in_hit_region != "no" and hit_self_complementarity == "no"):
        if( hit_end < istar_min):
            return "5p" 
        if( (hit_start+1) > istar_max):
            return "3p"     
    else:
        if(continuous_pairing == "no" and hit_self_complementarity == "yes"):
            return "discontinuous star strand and hit self complementarity"
        elif(continuous_pairing == "no"):
            return "discontinuous star strand"
        elif(hit_self_complementarity == "yes"):
            return "hit self complementarity"

    if(complementarity_in_hit_region == "no"):
        return "no complementarity in hit region"  
    print(hit_start, hit_end, istar_min, istar_max, continuous_pairing, complementarity_in_hit_region, hit_self_complementarity)

In [300]:
def get_star_start(hit_start, hit_end, values):
    c = 0
    i = hit_end - 3 - c
    while(values[i] == 0 and i >= 0):
        c += 1
        i = hit_end - 3 - c        
    if(values[i] - c < 1):                    
        return [max(values[i] - c,1), "negative value"]
    if(i < hit_start):
        return [values[i] - c, 'less than hit start']
    return [values[i] - c, '']

In [301]:
def get_star_end(hit_start, hit_end, values):
    if(hit_start - 2 >= 0 ):
        a = 0    
    else:
        a = abs(hit_start - 2)
    
    i = hit_start - 2 + a
    while(values[i] == 0 and i <= hit_end):
        a += 1
        i = hit_start - 2 + a
    
    if(i <= hit_end):
        return values[i] + a        
    return np.nan

In [302]:
def get_num_of_linking_residues(hit_start,hit_end, star_start, star_end, mir_type):
    if(mir_type == '5p'):
        return str(star_start - hit_end - 1)
    elif(mir_type == '3p'):
        return str(hit_start - star_end)
    elif(mir_type == "discontinuous star strand"):
        return "discontinuous star strand"
    elif(mir_type == "no complementarity in hit region"):
        print('error')    

In [303]:
def get_star_branching(star_start, star_end, star_range, values):    
    return not ((values[star_range-1] < star_start) | (values[star_range-1] > star_end)).all()

In [304]:
def getBOI_5p(hit_start, hit_end, values):
    # first calc latest non zero value
    for i in range(hit_end-1, 0, -1):
        if(values[i] != 0):
            last_v = values[i]
            last_i = i
            place = i
            break            
            
    for i in range(place-1, 0, -1):        
        v = values[i]
        if(v == 0):
            continue
        if(v < last_v):
            return [last_i + 1, last_v]                                                
        
        if((v - last_v) >= 3):
            s1 = set(range(last_v+1, v))
            s2 = set([values[i-1] for i in range(last_v+1, v)])
            if(len(s1.intersection(s2)) > 0):
                return [last_i + 1, last_v]    
        last_v = v            
        last_i = i            
    for i in range(0,hit_end):
        if(values[i] != 0 ):
            return [i + 1, values[i]]    
    print("Error")
                
def getBOI_3p(hit_start, hit_end, values):
    # first calc latest non zero value
    for i in range(hit_start, len(values)):    
        if(values[i] != 0):
            last_v = values[i]
            last_i = i
            place = i
            break            
            
    for i in range(place + 1, len(values)):
        v = values[i]
        if(v == 0):
            continue
        if(v > last_v):
            return [last_i + 1, last_v]                                                
        
        if((last_v - v) >= 3):
            s1 = set(range(v+1, last_v))
            s2 = set([values[i-1] for i in range(v+1, last_v)])
            if(len(s1.intersection(s2)) > 0):
                return [last_i + 1, last_v]    
        last_v = v            
        last_i = i            
    for i in range(hit_start, len(values)):
        if(values[i] != 0 ):
            return [i + 1, values[i]]    
    print("Error")            
    
    
def get_boi(hit_start, hit_end, values, mir_type):
    if(mir_type not in ['3p','5p']):
        return [np.nan, np.nan]
    if(mir_type == '5p'):
        return getBOI_5p(hit_start, hit_end, values)
    if(mir_type == '3p'):
        return getBOI_3p(hit_start, hit_end, values)

In [323]:
server_url = "http://jupyter.sysmanager.ir/tree/plant_microRNA_prediction"
def get_row(tag, path):
    #print(path)
    result = {}    
    ct = reformatCT(path)
    result['seq name'] = tag
    result['ct name'] = ""
    result['ct'] = f'=HYPERLINK("{server_url + path[1:]}")'
    result['pdf'] = f'=HYPERLINK("{server_url + path[1:-3] + ".pdf"}")'        
    [hit_start, hit_end, sign] = get_tag_info(tag)
    result['hit_start'] = hit_start + 1
    result['hit_end'] =  hit_end
    result['sign'] = sign
    dg = get_deltaG(ct)
    result['delta G'] = dg
    [nucleotide, index, values] = get_ct_data(ct)
    hit_seq = ''.join(nucleotide[hit_start:hit_end])
    result['hit_seq'] = hit_seq
    hit_range = index[hit_start:hit_end]
    hit_len = len(hit_range)
    result['hit_len'] = hit_len
    inc_srange = values[hit_start:hit_end] # Incomplete_Star_range

    complementarity_in_hit_region = get_complementarity_in_hit_region(inc_srange, hit_len)
    result['complementarity_in_hit_region'] = complementarity_in_hit_region    
    if(complementarity_in_hit_region == "no"):
        msg = "no complementarity in hit region"
        result['hit_self_complementarity'] = msg
        result['continuous_pairing'] = msg
        result['mir_type'] = msg
        result['star_start'] = msg
        result['star_end'] = msg
        result['star_seq'] = msg
        result['num_of_linking_residues'] = msg
        result['star_branching'] = msg
        result['boi_start']  = msg
        result['boi_end'] = msg
        return pd.Series(result) 
    
    hit_self_complementarity = get_hit_self_complementarity(hit_start, hit_end, inc_srange)
    result['hit_self_complementarity'] = hit_self_complementarity
    
    [istar_min, istar_max] = get_istar_min_max(inc_srange, hit_self_complementarity)  
    result['istar_min']  = istar_min
    result['istar_max']  = istar_max
    continuous_pairing = get_continuous_pairing(hit_start, hit_end, istar_min, istar_max, hit_self_complementarity)
    result['continuous_pairing'] = continuous_pairing    
    if(continuous_pairing == "no"):
        msg = "discontinuous star strand"        
        result['mir_type'] = msg
        result['star_start'] = msg
        result['star_end'] = msg
        result['star_seq'] = msg
        result['num_of_linking_residues'] = msg
        result['star_branching'] = msg
        result['boi_start']  = msg
        result['boi_end'] = msg
        return pd.Series(result) 
    
    mir_type = get_mir_type(hit_start, hit_end, istar_min, istar_max, continuous_pairing, complementarity_in_hit_region, hit_self_complementarity)
    result['mir_type'] = mir_type    
    if(mir_type not in ['3p', '5p']):
        msg = mir_type        
        result['star_start'] = msg
        result['star_end'] = msg
        result['star_seq'] = msg
        result['num_of_linking_residues'] = msg
        result['star_branching'] = msg
        result['boi_start']  = msg
        result['boi_end'] = msg
        return pd.Series(result) 
    
    [star_start, star_start_msg] = get_star_start(hit_start, hit_end, values)
    star_end = get_star_end(hit_start, hit_end, values)
    result['star_start'] = star_start 
    result['star_start_msg'] = star_start_msg     
    result['star_end'] = star_end    
    
    star_range = index[star_start - 1:star_end]
    star_seq = ''.join(nucleotide[star_start - 1:star_end])
    result['star_seq'] = star_seq
    num_of_linking_residues = get_num_of_linking_residues(hit_start,hit_end, star_start, star_end, mir_type)
    result['num_of_linking_residues'] = num_of_linking_residues
    star_branching = get_star_branching(star_start, star_end, star_range, values)
    result['star_branching'] = star_branching
    [boi_start, boi_end] = get_boi(hit_start, hit_end, values, mir_type)
    result['boi_start'] = boi_start
    result['boi_end'] =  boi_end
    return pd.Series(result)

In [324]:
'''def get_df_by_tag(tag):
    ct_files = glob.glob(f'{base}{reformat(tag)}/SEQ_*.ct')
    return pd.Series(ct_files).apply(lambda path: get_row(tag, path))
df['tag'].apply(lambda t : get_df_by_tag(t))''';

In [325]:
def get_df_by_tag(tag):
    ct_files = glob.glob(f'{base}{reformat(tag)}/SEQ_*.ct')
    return pd.Series(ct_files).apply(lambda path: get_row(tag, path))
dfs = []
for d in process_map(get_df_by_tag , df['tag'], tqdm_class=tqdm, max_workers=mp.cpu_count()- 1, chunksize=5):
    dfs.append(d)
df_result = pd.concat(dfs,axis=0)
df_result.to_csv("ct_analizer_result.csv", index=False)
!zip -r ct_analizer_result.csv.zip ./ct_analizer_result.csv

  0%|          | 0/200 [00:00<?, ?it/s]

updating: ct_analizer_result.csv (deflated 95%)


In [326]:
df_result

,seq name,ct name,ct,pdf,hit_start,hit_end,sign,delta G,hit_seq,hit_len,...,continuous_pairing,mir_type,star_start,star_end,star_seq,num_of_linking_residues,star_branching,boi_start,boi_end,star_start_msg
0,AMWY02000003.1|4085-4506|201-222|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,222,+,-100.01,ATAAAAACTTTTGAAAAATTCA,22,...,no,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,NaN
1,AMWY02000003.1|4085-4506|201-222|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,222,+,-98.95,ATAAAAACTTTTGAAAAATTCA,22,...,yes,5p,355,376,cgttcttcgacctcacgcattt,132,False,196,379,
2,AMWY02000003.1|4085-4506|201-222|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,222,+,-99.99,ATAAAAACTTTTGAAAAATTCA,22,...,yes,5p,221,418,CAtttttctctgtatgtctctttcattctattattgtccttccgat...,-2,True,213,228,
3,AMWY02000003.1|4085-4506|201-222|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,222,+,-96.88,ATAAAAACTTTTGAAAAATTCA,22,...,no,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,NaN
4,AMWY02000003.1|4085-4506|201-222|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,222,+,-101.61,ATAAAAACTTTTGAAAAATTCA,22,...,no,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,AMWY02000334.1|4703-5044|201-220|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,220,+,-125.23,TCGTTGAGGGCAGCGTTGAT,20,...,undifined,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,NaN
15,AMWY02000334.1|4703-5044|201-220|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,220,+,-121.85,TCGTTGAGGGCAGCGTTGAT,20,...,no,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,discontinuous star strand,NaN
16,AMWY02000334.1|4703-5044|201-220|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,220,+,-127.69,TCGTTGAGGGCAGCGTTGAT,20,...,yes,3p,167,206,caagctgtggccagctatggcaaagaaaagctcgTCGTTG,-6,True,208,197,
17,AMWY02000334.1|4703-5044|201-220|+,,"=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...","=HYPERLINK(""http://jupyter.sysmanager.ir/tree/...",201,220,+,-122.25,TCGTTGAGGGCAGCGTTGAT,20,...,undifined,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,hit self complementarity,NaN


# BLASTX or DIMOND

# DIAMOND

https://github.com/bbuchfink/diamond

In [ ]:
'''
import hashlib
import os
parallel = [os.path.join(dp, f) for dp, dn, filenames in os.walk("./PRNA_secondary_structure") for f in filenames ]
series = [os.path.join(dp, f) for dp, dn, filenames in os.walk("./SRNA_secondary_structure") for f in filenames ]
for i in range(len(parallel)):        
    md5_hash = hashlib.md5()
    with open(parallel[i],"rb") as file:        
        md5_hash.update(file.read())
        digest1 = md5_hash.hexdigest()                
    md5_hash = hashlib.md5()
    with open(series[i],"rb") as file:        
        md5_hash.update(file.read())
        digest2 = md5_hash.hexdigest()                
    if(digest1 != digest2):
        print(parallel[i])
        print(series[i])
        print("***********")     
'''